<a href="https://colab.research.google.com/github/oviyajay/ML-Final-Project/blob/main/MLfinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [144]:
import numpy as np
import tensorflow as tf
import os 
import cv2

#Members:...
#https://blog.paperspace.com/image-captioning-with-ai/
#image preprocessing



!git clone https://github.com/oviyajay/ML-Final-Project

#label the images
#create training data and test data 


fatal: destination path 'ML-Final-Project' already exists and is not an empty directory.


In [145]:
 
train_path = "/content/ML-Final-Project/maskdata/maskdata/train/"
os.listdir(train_path)

test_path = "/content/ML-Final-Project/maskdata/maskdata/test/"
os.listdir(test_path)



['with_mask', 'without_mask']

In [146]:
label_names = ['without_mask', 'with_mask']
IMG_SIZE = 50
LR = 1e-3
MODEL_NAME = 'withMaskvsnoMask-{}-{}.model'.format(LR, '6conv-basic')

def load_data(path):
  data = []

  for label in label_names:
    for filename in os.listdir(os.path.join(path, label)):
    
      img_file = os.path.join(path, label, filename)
       
      img = cv2.imread(img_file, cv2.IMREAD_COLOR)    
      

      # resizing the image for processing
      img = cv2.resize(img, (IMG_SIZE, IMG_SIZE), )
      

      #withmask: id = 1, withoutmask: id = 0
      label_id = label_names.index(label)
  

      # final step-forming the training data list with numpy array of the images 
      data.append([np.array(img), np.array(label_id)]) 

  return data



In [147]:
train_data = load_data(train_path)
test_data = load_data(test_path)


In [148]:
train_x = [i[0] for i in train_data]
train_x = np.array(train_x, dtype=float).reshape(-1, IMG_SIZE, IMG_SIZE, 3)  # num_image, width, height, channels
train_y = [i[1] for i in train_data]
train_y = np.array(train_y, dtype=float)
test_x = [i[0] for i in test_data]
test_x = np.array(test_x).reshape(-1, IMG_SIZE, IMG_SIZE, 3) 
test_y = [i[1] for i in test_data]
test_y = np.array(test_y, dtype=float)


In [149]:
#create our CNN Model
class MyModel(tf.keras.Model):
  def __init__(self):
    #define layers
    super(MyModel, self).__init__()
    self.conv1 = tf.keras.layers.Conv2D(32, kernel_size=(5,5),activation=tf.nn.relu, input_shape=(IMG_SIZE,IMG_SIZE,3))
    self.max1 = tf.keras.layers.MaxPooling2D(pool_size=(2,2), strides=(2,2))
    self.conv2 = tf.keras.layers.Conv2D(64, (5,5), activation = 'relu')
    self.max2=tf.keras.layers.MaxPooling2D(pool_size=(2,2))
  
  def call(self,inputs):
    #first convolution layer
    convOne = (self.conv1)(inputs)
    maxOne = (self.max1)(convOne)
    #second convolution layer
    convTwo = (self.conv2)(maxOne)
    maxTwo = (self.max2)(convTwo)
    #flatten the array
    flat = tf.keras.layers.Flatten()(maxTwo)
    #hidden layer
    hiddenLayer = tf.keras.layers.Dense(75, activation = 'relu')(flat) 
    #output layer
    outputLayer = tf.keras.layers.Dense(2, activation ='softmax')(hiddenLayer)



    return outputLayer

#create an instance of our model
model = MyModel()
#through keras, the model must be compiled by assiging it a loss function and metrics
#since we are using one-hot encoding to classify our images, we will use the loss function "Categorical Crossentropy"
model.compile(loss='sparse_categorical_crossentropy', metrics=['accuracy'], optimizer='sgd')
#next we must fit our data to our model and assign batch_size and number of epochs
history = model.fit(train_x,train_y, batch_size= 100, epochs=1000)
#_,accuracy = model.evaluate(train_x,train_y)




Train on 616 samples
Epoch 1/1000
616/616 [==============================] - 2s 4ms/sample - loss: 6057.5486 - acc: 0.4724
Epoch 2/1000
616/616 [==============================] - 2s 4ms/sample - loss: 0.7096 - acc: 0.4984
Epoch 3/1000
616/616 [==============================] - 2s 4ms/sample - loss: 0.6925 - acc: 0.5146
Epoch 4/1000
616/616 [==============================] - 2s 4ms/sample - loss: 0.6812 - acc: 0.5341
Epoch 5/1000
616/616 [==============================] - 2s 4ms/sample - loss: 0.6792 - acc: 0.5390
Epoch 6/1000
616/616 [==============================] - 2s 4ms/sample - loss: 0.6807 - acc: 0.5503
Epoch 7/1000
616/616 [==============================] - 2s 4ms/sample - loss: 0.6658 - acc: 0.5568
Epoch 8/1000
616/616 [==============================] - 2s 4ms/sample - loss: 0.6527 - acc: 0.5666
Epoch 9/1000
616/616 [==============================] - 2s 4ms/sample - loss: 0.6407 - acc: 0.5974
Epoch 10/1000
616/616 [==============================] - 2s 4ms/sample - loss: 0.6291

KeyboardInterrupt: ignored

In [ ]:
import matplotlib.pyplot as plt

plt.plot(model.evaluate(train_x,train_y))
plt.plot(model.evaluate(test_x,test_y))
plt.title('model')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

plt.plot(model.evaluate(train_x,train_y))
plt.plot(model.evaluate(test_x,test_y))
plt.plot(history.history['acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

plt.plot(model.evaluate(train_x,train_y))
plt.plot(model.evaluate(test_x,test_y))
plt.plot(history.history['loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()